In [1]:
%matplotlib inline

from time import sleep
from random import sample

import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

env = gym.make('CartPole-v0')

In [2]:
nactions =  2
nparams = 4
learning_rate = 0.05
gamma = .3

nepisode = 50

nbatch = 100
batch_legth = 512
batch_iter = 100


In [ ]:
_input = tf.placeholder(shape=[1, nparams], dtype=tf.float32)

# Hidden layer 1
weight = tf.Variable(tf.random_uniform([nparams, nactions],0 , 0.01))
bias = tf.Variable(tf.random_uniform([1, nactions], 0, 0.01))
Q = tf.add(tf.matmul(_input, weight), bias)
Q = tf.sigmoid(Q)

Qa = tf.argmax(Q, 1)

# weight_1 = tf.Variable(tf.random_uniform([nparams, 10],0 , 0.01))
# bias_1 = tf.Variable(tf.random_uniform([1, 10], 0, 0.01))
# H1 = tf.add(tf.matmul(_input, weight_1), bias_1)
# H1 = tf.tanh(H1)

# weight_2 = tf.Variable(tf.random_uniform([10, nactions],0 , 0.01))
# bias_2 = tf.Variable(tf.random_uniform([1, nactions], 0, 0.01))
# Q = tf.add(tf.matmul(H1, weight_2), bias_2)
# Q = tf.tanh(Q)
# action = tf.argmax(Q, 1)

Qe = tf.placeholder(shape=[1, nactions], dtype=tf.float32)
error = tf.reduce_sum(tf.square(Qe - Q))
trainer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error)

In [ ]:

beta = 0.5
init = tf.global_variables_initializer()
steps = list()
with tf.Session() as sess:
    sess.run(init)
    
    for episode in range(nepisode):
        print()
        # training: Collect play experience
        memos = list()
        done = False
        while len(memos) < batch_legth*batch_iter:
            done = False
            state = env.reset()
            while not done:
                rand_val = np.random.rand(1)
                res_action, res_Q = sess.run([Qa, Q], feed_dict={_input:[state]})
                new_state, reward, done, _ = env.step(res_action[0])
                memos.append( (state, res_action, reward, new_state, done) )
        print("\tCollect Step in Experience: ", len(memos))
        
        # training: Review playing experience
        for iteration in range(batch_iter):
            batch = sample(memos, batch_legth)
            for state, res_action, reward, new_state, done in batch:
                r = reward
                if not done:
                    next_Q = sess.run(Q, feed_dict={_input:[new_state]})
                    r = r + gamma*np.max(next_Q)
                else:
                    r = -100
                res_Q = sess.run(Q, feed_dict={_input:[state]})
                res_Q[0, res_action[0]] = r
                sess.run([trainer], feed_dict={_input:[state], Qe:res_Q})
            
                
    
        # playing
        state = env.reset()
        step = 0
        done = False
        while not done:
            env.render()
            action = sess.run([Qa], feed_dict={_input:[state]})
            action = action[0][0]
            state, reward, done, _ = env.step(action)
            step += 1
#         print(step)
        steps.append(step)
        print("in Episode {} agent can live in {} step".format(episode, step))
plt.plot(steps)
    


In [ ]:
# from time import sleep

# beta = 0.5


# init = tf.global_variables_initializer()

# steps = [0]

# with tf.Session() as sess:
#     sess.run(init)
#     for i in range(100):
#         state = env.reset()
#         done = False
#         step = 0
#         r = 0
#         print()
#         print(i)
#         print()
#         while done == False:
#             env.render()
            
#             rand_val = np.random.rand(1)
#             res_action, res_Q = sess.run([action, Q], feed_dict={_input:[state]})
#             # agent do something new
#             if  rand_val < beta:
#                 new_action = env.action_space.sample()
#                 res_action[0] = new_action
                
#             # do action 
#             new_state, reward, done, _ = env.step(res_action[0])
            
#             # cal reward
#             if done:
# #                 r = -100
#                 if step > max(steps):
#                     beta = beta*0.8
#                 else:
#                     r = -100
                
# #                 sleep()
# #                 break
#             else:
# #                 r = 0 + (1 if step > max(steps) else 0)
#                 r = step
#                 step += 1
            
#             # update weight
#             next_Q = sess.run(Q, feed_dict={_input:[new_state]})
#             res_Q[0, res_action[0]] = r + gamma*np.max(next_Q)
            
#             sess.run([trainer], feed_dict={_input:[state], Qe:res_Q})
#             print(step, res_action, res_Q, state, end=' ')
#             print('.' if rand_val < beta else '', end='')
#             print()
            
#             state = new_state
#         steps.append(step)
    
# plt.plot(steps)

In [ ]:
a = [[  8.58846959e-03  -9.89045802e-05]]
np.argmax(a)